In [29]:
import pandas as pd
docs = pd.read_excel("News_SGU_31077_Processed_1.xlsx")

In [30]:
from natasha import (
    NewsEmbedding,
    NewsNERTagger,
    Doc
)

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

def remove_proper_nouns(text):
    # Пропускаем не-строки и пустые строки
    if not isinstance(text, str) or not text.strip():
        return text
    
    try:
        doc = Doc(text)
        doc.tag_ner(ner_tagger)
        
        # Если нет сущностей, возвращаем исходный текст
        if not doc.spans:
            return text
            
        # Собираем сущности для удаления
        spans_to_remove = [span for span in doc.spans if span.type in ['PER', 'LOC', 'ORG']]
        
        # Если нечего удалять
        if not spans_to_remove:
            return text
            
        # Удаляем сущности (начиная с конца)
        text_cleaned = text
        for span in sorted(spans_to_remove, key=lambda x: x.start, reverse=True):
            text_cleaned = text_cleaned[:span.start] + text_cleaned[span.stop:]
            
        return text_cleaned.strip()
    
    except Exception as e:
        print(f"Ошибка при обработке текста: {text[:50]}... Ошибка: {str(e)}")
        return text  # В случае ошибки возвращаем исходный текст

In [34]:
docs['News_Tokens'] = docs["News_Tokens"].apply(lambda x: x.split(" "))
docs['News_Tokens'] = docs['News_Tokens'].apply(remove_proper_nouns)
docs.to_csv("data.csv", index=False)